## Pegagus

In [13]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch

In [14]:
src_text=["""WASHINGTON — There is, traditionally, a predictable rhythm to the start of a “much-awaited hearing” on Capitol Hill.

There are protesters and counterprotesters, enhanced police and news media presences, insufferable opening statements and an overall sense that something more momentous than usual might be going on.

The start of the Senate Judiciary Committee’s hearings for Judge Amy Coney Barrett on Monday had all of that. But like everything else in Washington these days, this was an entirely different kind of pinstripe rodeo.

Start with the hazmat suits.

“We are making a point,” said Jennie Spector, one of a few dozen demonstrators dressed in white hazmat suits convened outside the Hart Senate Office Building.

It was insane to be holding these hearings, Ms. Spector said. After all, two Republicans on the committee had recently tested positive for the coronavirus. And the committee chairman, Senator Lindsey Graham, the South Carolina Republican who was among those at the White House “super spreader” event last month, had refused to get tested in recent days. But there they all were. Showtime in a potential petri dish.

For the record, Ms. Spector said she purchased her hazmat suit at Walgreens. Apparently you can buy hazmat suits at Walgreens these days. How 2020 is that? The white jumpsuits doubled as decent rain gear in the morning drizzle.

Inside, everyone seemed to be remarking upon the senators’ varying degrees of public health diligence or recklessness. Viral vulnerability and co-morbidities were assessed. Senators Dianne Feinstein and Charles E. Grassley are 87!

As a general rule, U.S. Capitol Police officers have been diligent in their enforcement of mask requirements, with blatant exceptions. Senator Mike Lee, Republican of Utah, who attended the super spreader event — the Rose Garden announcement of President Trump’s nomination of Judge Barrett to the Supreme Court — and tested positive soon after, delivered his opening statement sans mask. He had previously assured everyone that he had been “cleared” by his doctor.
Editors’ Picks
‘S.N.L.’ Has a Theory About the Fly That Landed on Mike Pence’s Head
Man’s Best Friend Once Made Nice Wool Blankets, Too
A Black Belgian Student Saw a White Fraternity as His Ticket. It Was His Death.

Senator Ted Cruz, Republican of Texas, appeared remotely as he was self-quarantined after coming in contact with an infected party (Mr. Lee). Hand sanitizer, disinfecting machines and wipe dispensers are the new candy bowls in Senate offices.

Other novelties were more subtle, but telling. Upon gaveling the proceedings to order, Mr. Graham opened what he called “the hearing to confirm Judge Amy Coney Barrett to the Supreme Court,” a choice of words that pretty much gave away the game at the outset. Usually, the parliamentary term of art is for the chairman to commence a hearing to “consider the nomination” of so and so, not merely rubber stamp, or “confirm,” the nominee.

In other words, this would not be a deliberation as much as a media spectacle convened amid particles that may or may not be lethal."""]

In [7]:
src_text = [
    """ PG&E stated it scheduled the blackouts in response to forecasts for high winds amid dry conditions. The aim is to reduce the risk of wildfires. Nearly 800 thousand customers were scheduled to be affected by the shutoffs which were expected to last through at least midday tomorrow."""
]

In [18]:
model_name = 'google/pegasus-cnn_dailymail'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)

In [19]:
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)
batch = tokenizer.prepare_seq2seq_batch(src_text, truncation=True, padding='longest').to(torch_device)
translated = model.generate(**batch)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)

In [20]:
tgt_text

["The start of the Senate Judiciary Committee's hearings for Judge Amy Barrett on Monday had all of that.<n>But like everything else in Washington these days, this was an entirely different kind of pinstripe rodeo.<n>Two Republicans on the committee had recently tested positive for the coronavirus.<n>Hand sanitizer, disinfecting machines and wipe dispensers are the new candy bowls in Senate offices."]

## T5

In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

In [4]:
tgt_text

["California's largest electricity provider has turned off power to hundreds of thousands of customers."]

In [9]:
text = 'New Delhi, Apr 25 (PTI) Union minister Vijay Goel today batted for the unification of the three municipal corporations in the national capital saying a discussion over the issue was pertinent. The BJP leader, who was confident of a good show by his party in the MCD polls, the results of which will be declared tomorrow, said the civic bodies needed to be "revamped" in order to deliver the services to the people more effectively. The first thing needed was a discussion on the unification of the three municipal corporations and there should also be an end to the practice of sending Delhi government officials to serve in the civic bodies, said the Union Minister of State (Independent Charge) for Youth Affairs and Sports. "Barring one, the two other civic bodies have been incurring losses. It would be more fruitful and efficient if all the three were merged," he said, referring to the north, south and east Delhi municipal corporations. The erstwhile Municipal Corporation of Delhi (MCD) was trifurcated into NDMC, SDMC and EDMC by the then Sheila Dikshit-led Delhi government in 2012. Goel predicted a "thumping" victory for the BJP in the MCD polls. He said the newly-elected BJP councillors will be trained on the functioning of the civic bodies and dealing with the bureaucracy.'

In [10]:
text2 = 'PG&E stated it scheduled the blackouts in response to forecasts for high winds amid dry conditions. The aim is to reduce the risk of wildfires. Nearly 800 thousand customers were scheduled to be affected by the shutoffs which were expected to last through at least midday tomorrow.'

In [11]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('../weights/model_1_epochs', return_dict=True)
input_ids = tokenizer.encode(text2, return_tensors="pt")  # Batch size 1
outputs = model.generate(input_ids, max_length=150, 
                num_beams=2,
                repetition_penalty=2.5, 
                length_penalty=1.0, 
                early_stopping=True)

In [12]:
preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in outputs]

In [13]:
preds

['customers were expected to be affected by the blackouts which were scheduled to last through at least midday tomorrow.']

## Bart

In [1]:
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig

In [23]:
# see ``examples/summarization/bart/run_eval.py`` for a longer example
model = BartForConditionalGeneration.from_pretrained('/home/julien/data-science/document-summarization/src/weights/bart_large_cnn/')
tokenizer = BartTokenizer.from_pretrained('/home/julien/data-science/document-summarization/src/weights/bart_large_cnn/')

ARTICLE_TO_SUMMARIZE = src_text[0]# "My friends are cool but they eat too many carbs."
inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=1024, return_tensors='pt')

# Generate Summary
summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=5, early_stopping=True)
print([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids])

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


['The Senate']


In [ ]:
model.save_pretrained('/home/julien/data-science/document-summarization/src/weights')